In [1]:
!pip install faiss-cpu sentence-transformers langchain_chroma langchain_community nest_asyncio && pip install torch torchvision torchaudio && pip install ragatouille colbert && pip install ipywidgets && jupyter nbextension enable --py widgetsnbextension && pip install --upgrade chroma

  Obtaining dependency information for faiss-cpu from https://files.pythonhosted.org/packages/a6/9d/eaba10de74cd7fad91174a49481327eaf61fe80a2ad1e4ad16594256bf9d/faiss_cpu-1.9.0.post1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached faiss_cpu-1.9.0.post1-cp311-cp311-macosx_11_0_arm64.whl.metadata (4.4 kB)
  Obtaining dependency information for sentence-transformers from https://files.pythonhosted.org/packages/8b/c8/990e22a465e4771338da434d799578865d6d7ef1fdb50bd844b7ecdcfa19/sentence_transformers-3.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for langchain_chroma from https://files.pythonhosted.org/packages/bc/8c/ef0a589e523e72373a79a64cd1184682155113b865f889d815a9f264a755/langchain_chroma-0.1.4-py3-none-any.whl.metadata
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Obtaining dependency information for langchain_community from https://files.pythonhosted.org/packages/29/cd/bbee704e7634bac629c9adef37499f86fed30078cf98021d5ca4592c41

In [6]:
import os
import json
import logging
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from tqdm import tqdm
from fastapi import FastAPI, Query
from fastapi.responses import JSONResponse
import uvicorn

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize FastAPI app
app = FastAPI()

# Model configuration
MODEL_NAME = "BAAI/bge-m3"
embedding_function = HuggingFaceBgeEmbeddings(model_name=MODEL_NAME)

# Chroma persistent directory
PERSIST_DIR = "./chroma_db"

def get_text_collection(user_id: str):
    """
    Initializes a Chroma vector store for textual data using the provided user ID to create a unique collection name.
    """
    collection_name = f"{user_id}_text_collection"
    return Chroma(
        collection_name=collection_name,
        embedding_function=embedding_function,
         persist_directory=PERSIST_DIR,
    )
def embed_data():
    """
    Embeds all data from 'publications.json' into ChromaDB before the server starts.
    """
    try:
        # Load dataset
        with open("publications.json", "r") as file:
            data = json.load(file)

        # Initialize Chroma collection
        user_id = "default_user"
        logger.info(f"Initializing ChromaDB for user_id: {user_id}")
        text_store = get_text_collection(user_id)
        logger.info("ChromaDB vector store initialized successfully.")

        # Prepare and embed documents
        documents = []
        ids = []

        logger.info(f"Embedding {len(data)} documents from the JSON file...")

        for idx, item in enumerate(data):
            # Extract fields
            title = item.get("Title", f"Untitled Document {idx}")
            url = item.get("URL", "No URL")
            full_abstract = item.get("Full Abstract", "").strip()

            # Skip documents with insufficient data
            if not full_abstract and not title.strip():
                logger.warning(f"Skipping document {idx}: insufficient content.")
                continue

            # Combine title and abstract
            document_content = f"{title}\n{full_abstract or 'No additional content available.'}"

            # Create Document object
            document = Document(
                page_content=document_content,
                metadata={"title": title, "url": url},
            )
            documents.append(document)
            ids.append(f"doc_{idx}")

        # Store documents in the vector store
        logger.info("Storing documents in ChromaDB vector store...")
        text_store.add_documents(documents=documents, ids=ids)
        logger.info(f"Successfully embedded {len(documents)} documents.")
    except Exception as e:
        logger.error(f"Error embedding documents: {e}")
        logger.debug("Traceback:", exc_info=True)

@app.get("/")
async def read_root():
    """
    Root endpoint for testing.
    """
    return {"message": "Welcome to the ChromaDB Semantic Search API"}

@app.get("/search")
async def search(query: str, k: int = Query(default=5, description="Number of results to fetch")):
    """
    Performs a similarity search on the indexed data.
    """
    try:
        user_id = "default_user"  # Use the same user_id as in embed_data
        logger.info(f"Initializing ChromaDB for user_id: {user_id}")
        text_store = get_text_collection(user_id)
        logger.info("ChromaDB vector store initialized successfully.")

        # Perform similarity search
        logger.info(f"Received search query: {query}")
        results = text_store.similarity_search(query, k=k)

        # Format and log results
        formatted_results = []
        for idx, result in enumerate(results):
            title = result.metadata.get("title", "Unknown Title")
            url = result.metadata.get("url", "No URL")
            content = result.page_content
            logger.debug(f"Result {idx + 1}: Title: {title}, URL: {url}, Content: {content[:100]}...")
            formatted_results.append({"title": title, "url": url, "content": content})

        return {"results": formatted_results}
    except Exception as e:
        logger.error(f"Error during search: {e}")
        return JSONResponse(status_code=500, content={"message": "An error occurred during search."})

if __name__ == "__main__":
    # Embed data before starting the server
    embed_data()
    # Run FastAPI app
    uvicorn.run(app, host="127.0.0.1", port=8000)



INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps


KeyboardInterrupt: 

In [5]:
MODEL_NAME = "all-MiniLM-L6-v2"
embedding_function = HuggingFaceBgeEmbeddings(model_name=MODEL_NAME)

# Chroma persistent directory
PERSIST_DIR = "./chroma_db_final"
import torch
torch.set_default_device("cpu")

def get_text_collection(user_id: str):
    """
    Initializes a Chroma vector store for textual data using the provided user ID to create a unique collection name.
    """
    collection_name = f"{user_id}_text_collection"
    return Chroma(
        collection_name=collection_name,
        embedding_function=embedding_function,
         persist_directory=PERSIST_DIR,
    )


In [6]:
# verify the embedding
import os
embed_data()

NameError: name 'logger' is not defined

In [13]:
import os
import json
import logging
from langchain_core.documents import Document
from langchain_chroma import Chroma
import shutil

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def clear_system_cache():
    """
    Clears the system cache for the current user.
    """
    try:
        cache_path = os.path.expanduser("~/.cache/chroma")
        if os.path.exists(cache_path):
            shutil.rmtree(cache_path)
            logger.info("System cache cleared successfully.")
    except Exception as e:
        logger.error(f"Error clearing system cache: {e}")

def get_text_collection(user_id: str):
    """
    Initializes or retrieves a Chroma vector store for textual data using the provided user ID.
    """
    collection_name = f"{user_id}_text_collection"
    return Chroma(
        collection_name=collection_name,
        embedding_function=embedding_function,  # Use the global embedding_function
        persist_directory=PERSIST_DIR,
    )

def embed_data(batch_size=100):
    """
    Embeds all data from 'abstracts.json' into ChromaDB in smaller batches.
    """
    try:
        # Clear cache before starting
        clear_system_cache()
        
        file_path = "/Users/paniz/ReSearch/data/abstracts.json"
        if not os.path.exists(file_path):
            logger.error("Error: abstracts.json not found.")
            return

        with open(file_path, "r") as file:
            data = json.load(file)
            
        user_id = "default_user"
        text_store = get_text_collection(user_id)
        text_store.delete_collection()
        text_store = get_text_collection(user_id)
        
        logger.info("ChromaDB vector store initialized successfully.")

        total_docs = len(data)
        logger.info(f"Embedding {total_docs} documents in batches of {batch_size}...")

        for i in range(0, total_docs, batch_size):
            batch = data[i:i + batch_size]
            documents = []
            ids = []

            for idx, item in enumerate(batch):
                title = item.get("Title", "").strip()
                url = item.get("URL", "").strip()
                full_abstract = item.get("Full Abstract", "").strip()

                if not full_abstract:
                    logger.warning(f"Empty abstract for title: {title}")
                    continue

                logger.info(f"Abstract length for {title}: {len(full_abstract)}")

                document = Document(
                    page_content=full_abstract,
                    metadata={
                        "title": title,
                        "url": url,
                        "content_length": len(full_abstract)
                    }
                )
                documents.append(document)
                ids.append(f"doc_{i + idx}")

            if documents:
                logger.info(f"Batch {i // batch_size + 1}: Adding {len(documents)} documents")
                text_store.add_documents(documents=documents, ids=ids)

        logger.info("All batches embedded successfully.")
    except Exception as e:
        logger.error(f"Error embedding documents: {e}")

In [16]:

MODEL_NAME = "all-MiniLM-L6-v2"
embedding_function = HuggingFaceBgeEmbeddings(model_name=MODEL_NAME)
PERSIST_DIR = "./chroma_db_final"
embed_data()

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:__main__:ChromaDB vector store initialized successfully.
INFO:__main__:Embedding 2832 documents in batches of 100...
INFO:__main__:Abstract length for Computing 2002: 22
INFO:__main__:Abstract length for Approaching utopia: 290
INFO:__main__:Abstract length for Advances in Knowledge Discovery and Data Mining: 822
INFO:__main__:Abstract length for The Beckman report on database research: 161
INFO:__main__:Abstract length for Competitive generalized auctions: 1137
INFO:__main__:Abstract length for RAPID: 22
INFO:__main__:Abstract length for Implementation of an interpreter for a parallel language in Centaur: 22
INFO:__main__:Abstract length for The CAP filing

In [ ]:
import os
import json
import logging
from fastapi import FastAPI, Query
from fastapi.responses import JSONResponse
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
import uvicorn
import nest_asyncio

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# Model configuration
MODEL_NAME = "BAAI/bge-m3"
embedding_function = HuggingFaceBgeEmbeddings(model_name=MODEL_NAME)

# Chroma persistent directory
PERSIST_DIR = "./chroma_db_final"

def get_text_collection(user_id: str):
    """
    Initializes or retrieves a Chroma vector store for textual data using the provided user ID.
    """
    collection_name = f"{user_id}_text_collection"
    return Chroma(
        collection_name=collection_name,
        embedding_function=embedding_function,
        persist_directory=PERSIST_DIR,
    )

@app.get("/search_with_scores")
async def search_with_scores(
    query: str = Query(..., description="Search query text"),
    k: int = Query(5, description="Number of results to fetch")
):
    """
    Perform a similarity search for a given query and return results with similarity scores.
    Add suggestions for documents missing abstracts.
    """
    try:
        # Initialize Chroma collection
        user_id = "default_user"
        text_store = get_text_collection(user_id)

        # Perform similarity search with scores
        results = text_store.similarity_search_with_score(query, k=k)

        if not results:
            return {"results": [], "message": "No matching results found."}

        # Separate results with and without abstracts
        results_with_abstract = []
        suggestions = []

        for doc, score in results:
            if "Abstract not found" in doc.page_content or "failed to load" in doc.page_content:
                suggestions.append({
                    "title": doc.metadata.get("title", "Unknown Title"),
                    "url": doc.metadata.get("url", "No URL"),
                    "similarity_score": score
                })
            else:
                results_with_abstract.append({
                    "title": doc.metadata.get("title", "Unknown Title"),
                    "url": doc.metadata.get("url", "No URL"),
                    "content_snippet": doc.page_content[:100],  # Limit to 100 characters for snippet
                    "similarity_score": score
                })

        response = {"results": results_with_abstract}

        # Add suggestions if available
        if suggestions:
            response["suggestions"] = suggestions

        return response

    except Exception as e:
        logger.error(f"Error during similarity search: {e}")
        return JSONResponse(status_code=500, content={"message": "An error occurred during search."})

if __name__ == "__main__":
    # Ensure the persistent directory exists
    os.makedirs(PERSIST_DIR, exist_ok=True)
    
    # Run the FastAPI app
    uvicorn.run(app, host="127.0.0.1", port=8000)
